In [55]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 20: Race Condition ---</h2><p>The Historians are quite pixelated again. This time, a massive, black building looms over you - you're <a href="/2017/day/24">right outside</a> the CPU!</p>
<p>While The Historians get to work, a nearby program sees that you're idle and challenges you to a <em>race</em>. Apparently, you've arrived just in time for the frequently-held <em>race condition</em> festival!</p>
<p>The race takes place on a particularly long and twisting code path; programs compete to see who can finish in the <em>fewest picoseconds</em>. The <span title="If we give away enough mutexes, maybe someone will use one of them to fix the race condition!">winner</span> even gets their very own <a href="https://en.wikipedia.org/wiki/Lock_(computer_science)" target="_blank">mutex</a>!</p>
<p>They hand you a <em>map of the racetrack</em> (your puzzle input). For example:</p>
<pre><code>###############
#...#...#.....#
#.#.#.#.#.###.#
#<em>S</em>#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..<em>E</em>#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>The map consists of track (<code>.</code>) - including the <em>start</em> (<code>S</code>) and <em>end</em> (<code>E</code>) positions (both of which also count as track) - and <em>walls</em> (<code>#</code>).</p>
<p>When a program runs through the racetrack, it starts at the start position. Then, it is allowed to move up, down, left, or right; each such move takes <em>1 picosecond</em>. The goal is to reach the end position as quickly as possible. In this example racetrack, the fastest time is <code>84</code> picoseconds.</p>
<p>Because there is only a single path from the start to the end and the programs all go the same speed, the races used to be pretty boring. To make things more interesting, they introduced a new rule to the races: programs are allowed to <em>cheat</em>.</p>
<p>The rules for cheating are very strict. <em>Exactly once</em> during a race, a program may <em>disable collision</em> for up to <em>2 picoseconds</em>. This allows the program to <em>pass through walls</em> as if they were regular track. At the end of the cheat, the program must be back on normal track again; otherwise, it will receive a <a href="https://en.wikipedia.org/wiki/Segmentation_fault" target="_blank">segmentation fault</a> and get disqualified.</p>
<p>So, a program could complete the course in 72 picoseconds (saving <em>12 picoseconds</em>) by cheating for the two moves marked <code>1</code> and <code>2</code>:</p>
<pre><code>###############
#...#...12....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>Or, a program could complete the course in 64 picoseconds (saving <em>20 picoseconds</em>) by cheating for the two moves marked <code>1</code> and <code>2</code>:</p>
<pre><code>###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...12..#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>This cheat saves <em>38 picoseconds</em>:</p>
<pre><code>###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.####1##.###
#...###.2.#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>This cheat saves <em>64 picoseconds</em> and takes the program directly to the end:</p>
<pre><code>###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..21...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>Each cheat has a distinct <em>start position</em> (the position where the cheat is activated, just before the first move that is allowed to go through walls) and <em>end position</em>; cheats are uniquely identified by their start position and end position.</p>
<p>In this example, the total number of cheats (grouped by the amount of time they save) are as follows:</p>
<ul>
<li>There are 14 cheats that save 2 picoseconds.</li>
<li>There are 14 cheats that save 4 picoseconds.</li>
<li>There are 2 cheats that save 6 picoseconds.</li>
<li>There are 4 cheats that save 8 picoseconds.</li>
<li>There are 2 cheats that save 10 picoseconds.</li>
<li>There are 3 cheats that save 12 picoseconds.</li>
<li>There is one cheat that saves 20 picoseconds.</li>
<li>There is one cheat that saves 36 picoseconds.</li>
<li>There is one cheat that saves 38 picoseconds.</li>
<li>There is one cheat that saves 40 picoseconds.</li>
<li>There is one cheat that saves 64 picoseconds.</li>
</ul>
<p>You aren't sure what the conditions of the racetrack will be like, so to give yourself as many options as possible, you'll need a list of the best cheats. <em>How many cheats would save you at least 100 picoseconds?</em></p>
</article>


In [56]:
from collections import deque
from collections.abc import Iterator
from scipy.spatial.distance import cityblock

from more_itertools import first


tests = [
    {
        "name": "Example",
        "s": """
            ###############
            #...#...#.....#
            #.#.#.#.#.###.#
            #S#...#.#.#...#
            #######.#.#.###
            #######.#.#...#
            #######.#.###.#
            ###..E#...#...#
            ###.#######.###
            #...###...#...#
            #.#####.#.###.#
            #.#...#.#.#...#
            #.#.#.#.#.#.###
            #...#...#...###
            ###############
        """,
        "min_saving": 12,
        "expected": sum(
            v
            for k, v in {
                2: 14,
                4: 14,
                6: 2,
                8: 4,
                10: 2,
                12: 3,
                20: 1,
                36: 1,
                38: 1,
                40: 1,
                64: 1,
            }.items()
            if k >= 12
        ),
    },
]


class RaceTrack(Str):
    def __init__(self, s: str) -> None:
        self.map = [list(l.strip()) for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.map), len(self.map[0])

    def savings_of_at_least_slow(self, min_saving: int) -> int:
        path = self.shortest_path()
        savings = 0

        for left in range(len(path)):
            for right in range(left + 1, len(path)):
                row_left, col_left = path[left]
                row_right, col_right = path[right]
                row_mid = abs(row_right - row_left) // 2
                col_mid = abs(col_right - col_left) // 2

                if (
                    right - left - 2 > 0
                    and self.map[row_mid][col_mid] == "#"
                    and self.distance(path, left, right) == 2
                ):
                    saving = right - left - 2
                    if saving >= min_saving:
                        savings += 1

        return savings

    def distance(self, path, left, right):
        return abs(path[left][0] - path[right][0]) + abs(path[left][1] - path[right][1])

    def savings_of_at_least(self, min_saving: int) -> int:
        path = self.shortest_path()
        cell_path_index = {c: i for i, c in enumerate(path)}
        count = 0

        for i, (left_r, left_c) in enumerate(path):
            for right_r, right_c in self.neigbor_other_site_of_wall(left_r, left_c):
                j = cell_path_index.get((right_r, right_c), -inf)
                if j > i and j - i - 2 >= min_saving:
                    count += 1

        return count

    def shortest_path(self) -> list[tuple[int, int]]:
        queue = deque([(*self.get_start(), [])])

        while queue:
            row, col, path = queue.popleft()

            path = path + [(row, col)]

            if self.map[row][col] == "E":
                return path

            queue.extend(
                (r, c, path)
                for r, c in self.neighbors(row, col)
                if len(path) < 2 or (r, c) != path[-2]
            )

    def neigbor_other_site_of_wall(
        self, row: int, col: int
    ) -> Iterator[tuple[int, int]]:
        seen = {(row, col)}
        for r, c in self.neighbors(row, col, cheat_mode=True):
            if self.map[r][c] == "#":
                for rr, cc in self.neighbors(r, c):
                    if (rr, cc) not in seen:
                        seen.add((rr, cc))
                        yield rr, cc

    def neighbors(
        self, row: int, col: int, cheat_mode=False
    ) -> Iterator[tuple[int, int]]:
        dirs = (-1, 0), (0, 1), (1, 0), (0, -1)
        return (
            (row + dr, col + dc)
            for dr, dc in dirs
            if 0 < row + dr < self.rows
            and 0 <= col + dc < self.cols
            and (cheat_mode or self.map[row + dr][col + dc] != "#")
        )

    def get_start(self) -> tuple[int, int]:
        return first(
            (r, c)
            for r, c in product(range(self.rows), range(self.cols))
            if self.map[r][c] == "S"
        )


@test(tests=tests[:])
def partI_test(s: str, min_saving: int) -> int:
    r = RaceTrack(s)
    return r.savings_of_at_least(min_saving)


Test Example passed, for partI_test.
Success


In [57]:
with open("../input/day20.txt") as f:
    puzzle = f.read()

print(f"Part I: { RaceTrack(puzzle).savings_of_at_least(100) }")

Part I: 1497


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1497</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The programs seem perplexed by your list of cheats. Apparently, the two-picosecond cheating rule was deprecated several milliseconds ago! The latest version of the cheating rule permits a single cheat that instead lasts at most <em>20 picoseconds</em>.</p>
<p>Now, in addition to all the cheats that were possible in just two picoseconds, many more cheats are possible. This six-picosecond cheat saves <em>76 picoseconds</em>:</p>
<pre><code>###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#1#####.#.#.###
#2#####.#.#...#
#3#####.#.###.#
#456.E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>Because this cheat has the same start and end positions as the one above, it's the <em>same cheat</em>, even though the path taken during the cheat is different:</p>
<pre><code>###############
#...#...#.....#
#.#.#.#.#.###.#
#S12..#.#.#...#
###3###.#.#.###
###4###.#.#...#
###5###.#.###.#
###6.E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
</code></pre>
<p>Cheats don't need to use all 20 picoseconds; cheats can last any amount of time up to and including 20 picoseconds (but can still only end when the program is on normal track). Any cheat time not used is lost; it can't be saved for another cheat later.</p>
<p>You'll still need a list of the best cheats, but now there are even more to choose between. Here are the quantities of cheats in this example that save <em>50 picoseconds or more</em>:</p>
<ul>
<li>There are 32 cheats that save 50 picoseconds.</li>
<li>There are 31 cheats that save 52 picoseconds.</li>
<li>There are 29 cheats that save 54 picoseconds.</li>
<li>There are 39 cheats that save 56 picoseconds.</li>
<li>There are 25 cheats that save 58 picoseconds.</li>
<li>There are 23 cheats that save 60 picoseconds.</li>
<li>There are 20 cheats that save 62 picoseconds.</li>
<li>There are 19 cheats that save 64 picoseconds.</li>
<li>There are 12 cheats that save 66 picoseconds.</li>
<li>There are 14 cheats that save 68 picoseconds.</li>
<li>There are 12 cheats that save 70 picoseconds.</li>
<li>There are 22 cheats that save 72 picoseconds.</li>
<li>There are 4 cheats that save 74 picoseconds.</li>
<li>There are 3 cheats that save 76 picoseconds.</li>
</ul>
<p>Find the best cheats using the updated cheating rules. <em>How many cheats would save you at least 100 picoseconds?</em></p>
</article>


In [58]:
from typing import override


tests = [
    {
        "name": "Example",
        "s": """
            ###############
            #...#...#.....#
            #.#.#.#.#.###.#
            #S#...#.#.#...#
            #######.#.#.###
            #######.#.#...#
            #######.#.###.#
            ###..E#...#...#
            ###.#######.###
            #...###...#...#
            #.#####.#.###.#
            #.#...#.#.#...#
            #.#.#.#.#.#.###
            #...#...#...###
            ###############
        """,
        "min_saving": 50,
        "expected": 285,
    },
]


class RaceTrackUpdate(RaceTrack):
    @override
    def savings_of_at_least_slow(self, min_saving: int) -> int:
        path = self.shortest_path()
        savings = 0
        for left, (left_r, left_c) in enumerate(path):
            for right in range(left + min_saving + 2, len(path)):
                right_r, right_c = path[right]
                cb = abs(right_r - left_r) + abs(left_c - right_c)

                if right - left - cb >= min_saving and 2 <= cb <= 20:
                    savings += 1

        return savings

    @override
    def savings_of_at_least(self, min_saving: int) -> int:
        path = self.shortest_path()
        savings = 0
        cell_path_index = {c: i for i, c in enumerate(path)}

        for left, (row, col) in enumerate(path):
            for distance, r, c in self.all_cells_between(row, col):
                right = cell_path_index[(r, c)]
                if right - left - distance >= min_saving:
                    savings += 1

        return savings

    def all_cells_between(self, row: int, col: int) -> Iterator[tuple[int, int]]:
        queue = deque([(row, col)])
        distance = 0
        seen = set()

        while queue:
            if distance > 20:
                return

            for _ in range(len(queue)):
                r, c = queue.popleft()

                if not (0 <= r < self.rows and 0 <= c < self.cols):
                    continue

                if (r, c) in seen:
                    continue

                seen.add((r, c))

                if distance >= 2 and self.map[r][c] != "#":
                    yield distance, r, c

                queue.extend(self.neighbors(r, c, True))

            distance += 1


@test(tests=tests)
def partII_test(s: str, min_saving: int) -> int:
    r = RaceTrackUpdate(s)
    d = r.savings_of_at_least_slow(min_saving)
    return d


Test Example passed, for partII_test.
Success


In [59]:
print(f"Part II: { RaceTrackUpdate(puzzle).savings_of_at_least_slow(100) }")

Part II: 1030809


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1030809</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


In [63]:
[(i1, i2) for i1, i2 in product(range(1, 21), repeat=2) if i2 + i1 >= 100]

[]